In [32]:
import select
import psycopg2
import psycopg2.extensions

field_names =  ['user_screen_name', 'user_name', 'id_str', 'created_at', 
                'sentiment', 'categories', 'text', 'klout_score', 'segment_id',
               'got_reply', 'followers_count']

segment_dict = {0:'-', 1:'Mass Market', 2:'Young Professional',
                3:'Mass Afluent', 4:'Affluent', 5:'High Net Worth'}

def get_priority_key(tw_l):
    tw_l[0]['segment_id']/2 + 

In [ ]:
from flask import Flask, render_template, jsonify
from flask_sockets import Sockets
from json import dumps
import gevent
import time
from gevent.pool import Pool


app = Flask(__name__)
sockets = Sockets(app)

@sockets.route('/twitter_stream')
def twitter_stream(ws):
    j = 0
    
    conn = psycopg2.connect(database="twitter", user="postgres")
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()
    curs.execute("LISTEN new_tweet;")
    
    while not ws.closed:
        conn.poll()
        t0 = time.time()
        if conn.notifies:
            notify = conn.notifies.pop(0)
            
            ws.send("{notification:\'new tweet\'}")
            j+=1
        if time.time() - t0 > 3600:
            print("1hr timeout")
            break
        else:
            gevent.sleep()
    conn.close()

@app.route('/data', methods=['GET'])
def provide_data():
    #https://twitter.com/statuses/ID
    conn = psycopg2.connect(database="twitter", user="postgres")
    conn.set_isolation_level(psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT)
    curs = conn.cursor()

    curs.execute("""select stream.user_screen_name, stream.user_name, stream.id_str,
                stream.created_at, stream.sentiment, stream.categories, stream.text,
                customers.klout_score, customers.segment, customers.got_reply, customers.followers_count
                from stream
                INNER JOIN customers
                ON customers.user_screen_name=stream.user_screen_name
                where stream.user_screen_name in 
                (
                    select tmp.user_screen_name
                    from (
                        select user_screen_name as user_screen_name, max(id_str) as max_id_str
                        from stream
                        group by user_screen_name
                    ) tmp
                    left join (
                        select user_screen_name, got_reply
                        from customers
                    ) tmp2
                    on tmp.user_screen_name=tmp2.user_screen_name
                    order by tmp2.got_reply asc, tmp.max_id_str desc limit 10
                )
                order by customers.got_reply desc, stream.id_str desc
                ;""")
    rec = curs.fetchall()
    conn.close()
    
    user_blocks = {v[0]:[] for v in rec}
    for row in rec:
        if len(user_blocks[row[0]]) < 5:
            user_blocks[row[0]] += [{k:v for k,v in zip(field_names, row)}]
            user_blocks[row[0]][-1]['id_str'] = str(user_blocks[row[0]][-1]['id_str'])
            user_blocks[row[0]][-1]['segment'] = segment_dict[user_blocks[row[0]][-1]['segment_id']]
    
    return jsonify({'data':user_blocks.values()})

@app.route('/')
def homepage():
    try:
        return render_template("index.html")
    except Exception, e:
        return str(e)

if __name__ == "__main__":
    from gevent import pywsgi
    from geventwebsocket.handler import WebSocketHandler
    server = pywsgi.WSGIServer(('', 8090), app, handler_class=WebSocketHandler, spawn=Pool())
    server.serve_forever()

In [ ]:
df